In [1]:
#!/usr/bin/env python
# coding: utf-8
import sys
import scanpy as sc
import anndata
import pandas as pd
import numpy as np
import os
import gc
import decoupler as dc
import regex as re
os.chdir("/Volumes/RicoData2/ReHeat2/raw")

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [2]:
sc_dat = sc.read_h5ad(filename = "./Kuppe2022_MI/local.h5ad")
sc_dat.obs["barcode"] = sc_dat.obs.index.values

In [3]:
sc_dat.X = sc_dat.raw.X

In [4]:
sc_dat.X.sum(1)

matrix([[ 2597.],
        [12995.],
        [ 6361.],
        ...,
        [  808.],
        [  862.],
        [  511.]], dtype=float32)

In [5]:
sc_dat.obs.columns

Index(['sample', 'n_counts', 'n_genes', 'percent_mito', 'doublet_score',
       'dissociation_score', 'cell_type_original', 'patient_region_id',
       'donor_id', 'patient_group', 'major_labl', 'final_cluster',
       'assay_ontology_term_id', 'development_stage_ontology_term_id',
       'disease_ontology_term_id', 'self_reported_ethnicity_ontology_term_id',
       'is_primary_data', 'organism_ontology_term_id', 'sex_ontology_term_id',
       'tissue_ontology_term_id', 'cell_type_ontology_term_id',
       'suspension_type', 'cell_type', 'assay', 'disease', 'organism', 'sex',
       'tissue', 'self_reported_ethnicity', 'development_stage', 'barcode'],
      dtype='object')

In [6]:
disease_df = {'major_labl': ['CTRL', 'FZ','BZ','IZ','RZ'],
              'disease_code': ["NF","MI_FZ","MI_BZ","MI_IZ", "MI_RZ"],
              'heart_failure': ["NF", "HF", "HF", "HF", "HF"]}

disease_df = pd.DataFrame(disease_df)

In [7]:
new_codes = sc_dat.obs.merge(disease_df, on='major_labl', 
                           how='left')[["barcode","major_labl","disease_code","heart_failure"]]
new_codes.set_index("barcode", inplace = True)
new_codes = new_codes.loc[sc_dat.obs.index.values, :]
sc_dat.obs["disease_code"] = new_codes["disease_code"].values
sc_dat.obs["heart_failure"] = new_codes["heart_failure"].values

In [8]:
sc_dat.obs[["major_labl","disease_code","heart_failure"]].drop_duplicates()

,major_labl,disease_code,heart_failure
AAACCCACAAAGGAGA-1_1_1_1_1_1_1_1_1_1_1_1_1_1_1,CTRL,NF,NF
AAACCCACAAGTGGCA-1_2_1_1_1_1_1_1_1_1_1_1_1_1_1,FZ,MI_FZ,HF
AAACCCAAGGAACGAA-1_1_1_1_1_1_1_1_1_1_1_1_1_1,BZ,MI_BZ,HF
AAACGCTTCCCTCATG-1_2_1_1_1_1_1_1_1_1_1_1_1_1,IZ,MI_IZ,HF
AAACCCAAGCGAAACC-1_1_1_1_1_1_1_1_1_1_1_1,RZ,MI_RZ,HF


In [9]:
available_cells = sc_dat.obs[["cell_type_original"]].drop_duplicates()
available_cells

,cell_type_original
AAACCCACAAAGGAGA-1_1_1_1_1_1_1_1_1_1_1_1_1_1_1,Fibroblast
AAACCCACAGAGATGC-1_1_1_1_1_1_1_1_1_1_1_1_1_1_1,Cardiomyocyte
AAACCCAGTGCTTATG-1_1_1_1_1_1_1_1_1_1_1_1_1_1_1,Endothelial
AAACCCATCACGAGGA-1_1_1_1_1_1_1_1_1_1_1_1_1_1_1,Myeloid
AAACGCTCAGAACTTC-1_1_1_1_1_1_1_1_1_1_1_1_1_1_1,Neuronal
AAACGCTTCACTCACC-1_1_1_1_1_1_1_1_1_1_1_1_1_1_1,Pericyte
AAAGGATTCGTTCAGA-1_1_1_1_1_1_1_1_1_1_1_1_1_1_1,Mast
AACCAACCATTGGGAG-1_1_1_1_1_1_1_1_1_1_1_1_1_1_1,Lymphoid
AACCACAGTGGAAGTC-1_1_1_1_1_1_1_1_1_1_1_1_1_1_1,Cycling cells
AACCTTTCAGGAAGTC-1_1_1_1_1_1_1_1_1_1_1_1_1_1_1,vSMCs


In [10]:
available_cells["cell_type_uni"] = "none"
available_cells

,cell_type_original,cell_type_uni
AAACCCACAAAGGAGA-1_1_1_1_1_1_1_1_1_1_1_1_1_1_1,Fibroblast,none
AAACCCACAGAGATGC-1_1_1_1_1_1_1_1_1_1_1_1_1_1_1,Cardiomyocyte,none
AAACCCAGTGCTTATG-1_1_1_1_1_1_1_1_1_1_1_1_1_1_1,Endothelial,none
AAACCCATCACGAGGA-1_1_1_1_1_1_1_1_1_1_1_1_1_1_1,Myeloid,none
AAACGCTCAGAACTTC-1_1_1_1_1_1_1_1_1_1_1_1_1_1_1,Neuronal,none
AAACGCTTCACTCACC-1_1_1_1_1_1_1_1_1_1_1_1_1_1_1,Pericyte,none
AAAGGATTCGTTCAGA-1_1_1_1_1_1_1_1_1_1_1_1_1_1_1,Mast,none
AACCAACCATTGGGAG-1_1_1_1_1_1_1_1_1_1_1_1_1_1_1,Lymphoid,none
AACCACAGTGGAAGTC-1_1_1_1_1_1_1_1_1_1_1_1_1_1_1,Cycling cells,none
AACCTTTCAGGAAGTC-1_1_1_1_1_1_1_1_1_1_1_1_1_1_1,vSMCs,none


In [11]:
available_cells.loc[available_cells["cell_type_original"].str.contains('Fibroblast'), 'cell_type_uni'] = "Fib"
available_cells.loc[available_cells["cell_type_original"].str.contains('Myeloid'), 'cell_type_uni'] = "Myeloid"
available_cells.loc[available_cells["cell_type_original"].str.contains('Mast'), 'cell_type_uni'] = "Myeloid"
available_cells.loc[available_cells["cell_type_original"].str.contains('Cardiomyocyte'), 'cell_type_uni'] = "CM"
available_cells.loc[available_cells["cell_type_original"].str.contains('Endothelial'), 'cell_type_uni'] = "Endo"
available_cells.loc[available_cells["cell_type_original"].str.contains('vSMCs'), 'cell_type_uni'] = "vSMCs"
available_cells.loc[available_cells["cell_type_original"].str.contains('Lymphoid'), 'cell_type_uni'] = "Lymphoid"
available_cells.loc[available_cells["cell_type_original"].str.contains('Pericyte'), 'cell_type_uni'] = "PC"

In [12]:
available_cells

,cell_type_original,cell_type_uni
AAACCCACAAAGGAGA-1_1_1_1_1_1_1_1_1_1_1_1_1_1_1,Fibroblast,Fib
AAACCCACAGAGATGC-1_1_1_1_1_1_1_1_1_1_1_1_1_1_1,Cardiomyocyte,CM
AAACCCAGTGCTTATG-1_1_1_1_1_1_1_1_1_1_1_1_1_1_1,Endothelial,Endo
AAACCCATCACGAGGA-1_1_1_1_1_1_1_1_1_1_1_1_1_1_1,Myeloid,Myeloid
AAACGCTCAGAACTTC-1_1_1_1_1_1_1_1_1_1_1_1_1_1_1,Neuronal,none
AAACGCTTCACTCACC-1_1_1_1_1_1_1_1_1_1_1_1_1_1_1,Pericyte,PC
AAAGGATTCGTTCAGA-1_1_1_1_1_1_1_1_1_1_1_1_1_1_1,Mast,Myeloid
AACCAACCATTGGGAG-1_1_1_1_1_1_1_1_1_1_1_1_1_1_1,Lymphoid,Lymphoid
AACCACAGTGGAAGTC-1_1_1_1_1_1_1_1_1_1_1_1_1_1_1,Cycling cells,none
AACCTTTCAGGAAGTC-1_1_1_1_1_1_1_1_1_1_1_1_1_1_1,vSMCs,vSMCs


In [13]:
# Generates a new dataframe that keeps the new cell-type annotation
new_cts = sc_dat.obs.merge(available_cells, on='cell_type_original', 
                           how='left')[["barcode","cell_type_uni"]]
new_cts.set_index("barcode", inplace = True)
new_cts = new_cts.loc[sc_dat.obs.index.values, :]
sc_dat.obs["cell_type_uni"] = new_cts["cell_type_uni"].values

In [14]:
sc_dat.obs[["cell_type_original", "cell_type_uni"]].drop_duplicates()

,cell_type_original,cell_type_uni
AAACCCACAAAGGAGA-1_1_1_1_1_1_1_1_1_1_1_1_1_1_1,Fibroblast,Fib
AAACCCACAGAGATGC-1_1_1_1_1_1_1_1_1_1_1_1_1_1_1,Cardiomyocyte,CM
AAACCCAGTGCTTATG-1_1_1_1_1_1_1_1_1_1_1_1_1_1_1,Endothelial,Endo
AAACCCATCACGAGGA-1_1_1_1_1_1_1_1_1_1_1_1_1_1_1,Myeloid,Myeloid
AAACGCTCAGAACTTC-1_1_1_1_1_1_1_1_1_1_1_1_1_1_1,Neuronal,none
AAACGCTTCACTCACC-1_1_1_1_1_1_1_1_1_1_1_1_1_1_1,Pericyte,PC
AAAGGATTCGTTCAGA-1_1_1_1_1_1_1_1_1_1_1_1_1_1_1,Mast,Myeloid
AACCAACCATTGGGAG-1_1_1_1_1_1_1_1_1_1_1_1_1_1_1,Lymphoid,Lymphoid
AACCACAGTGGAAGTC-1_1_1_1_1_1_1_1_1_1_1_1_1_1_1,Cycling cells,none
AACCTTTCAGGAAGTC-1_1_1_1_1_1_1_1_1_1_1_1_1_1_1,vSMCs,vSMCs


In [15]:
sc_dat = sc_dat[sc_dat.obs[["cell_type_uni"]].values != "none",]

In [16]:
# Filter obs to contain things that are relevant
sc_dat.obs = sc_dat.obs[['patient_region_id',
                         'cell_type_uni',
                         'donor_id',
                         'patient_group',
                         'major_labl',
                         'sex',
                         'disease_code',
                        'heart_failure']]

In [17]:
sc_dat.obs = sc_dat.obs.rename(columns={"patient_region_id": "sample_id", "cell_type_uni":"cell_type"})

In [18]:
sc_dat.obs

,sample_id,cell_type,donor_id,patient_group,major_labl,sex,disease_code,heart_failure
AAACCCACAAAGGAGA-1_1_1_1_1_1_1_1_1_1_1_1_1_1_1,control_P1,Fib,P1,myogenic,CTRL,male,NF,NF
AAACCCACAGAGATGC-1_1_1_1_1_1_1_1_1_1_1_1_1_1_1,control_P1,CM,P1,myogenic,CTRL,male,NF,NF
AAACCCAGTAACGGTG-1_1_1_1_1_1_1_1_1_1_1_1_1_1_1,control_P1,CM,P1,myogenic,CTRL,male,NF,NF
AAACCCAGTCCAGAAG-1_1_1_1_1_1_1_1_1_1_1_1_1_1_1,control_P1,Fib,P1,myogenic,CTRL,male,NF,NF
AAACCCAGTCGTCGGT-1_1_1_1_1_1_1_1_1_1_1_1_1_1_1,control_P1,CM,P1,myogenic,CTRL,male,NF,NF
...,...,...,...,...,...,...,...,...
TTTCATGGTCTTGCTC-1,IZ_P16,Fib,P16,ischemic,IZ,male,MI_IZ,HF
TTTCCTCAGGTTCATC-1,IZ_P16,Lymphoid,P16,ischemic,IZ,male,MI_IZ,HF
TTTCGATTCCGGTAAT-1,IZ_P16,Myeloid,P16,ischemic,IZ,male,MI_IZ,HF
TTTGACTTCGCCAACG-1,IZ_P16,Endo,P16,ischemic,IZ,male,MI_IZ,HF


In [19]:
sc_dat.write_h5ad("/Volumes/RicoData2/ReHeat2/simplified/Kuppe2022_MI.h5ad")